In [10]:
import tensorflow as tf

In [11]:
# Import TensorFlow Datasets
import tensorflow_datasets as tfds
tfds.disable_progress_bar()

In [12]:
# Helper libraries
import math
import numpy as np
import matplotlib.pyplot as plt

In [13]:
# the wonderful logging
import logging
logger = tf.get_logger()
logger.setLevel(logging.ERROR)